# **Tutorial** - Topic Modeling with BERTopic
(last updated 08-06-2021)

In this tutorial we will be exploring how to use BERTopic to create topics from the well-known 20Newsgroups dataset. The most frequent use-cases and methods are discussed together with important parameters to keep a look out for. 


## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [1]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

In [1]:
from sentence_transformers import SentenceTransformer


In [2]:
cn_dense_retrieval_model = SentenceTransformer('junnyu/roformer_chinese_sim_char_ft_base')

Some weights of the model checkpoint at /home/r09944035/.cache/torch/sentence_transformers/junnyu_roformer_chinese_sim_char_ft_base were not used when initializing RoFormerModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'roformer.pooler.dense.bias', 'roformer.pooler.dense.weight']
- This IS expected if you are initializing RoFormerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RoFormerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RoFormerModel were 

# Data
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts

In [1]:
from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

In [6]:
docs = docs[:200]

In [6]:
import sys
from paddlenlp import Taskflow
sys.path.insert(1, '../../memory_model_app/')
seg = Taskflow('word_segmentation')
from zh_vectorizer_model import ChineseVectorizerModel
vectorizer_model = ChineseVectorizerModel(seg)

In [53]:
docs = list()
with open('/home/r09944035/ZenboDev_MemoryQA/topicModeling/synthetic_memories/generation/output_20000.txt','r',encoding='utf-8') as file:
    docs = file.readlines()

In [54]:
len(docs)

24795

In [55]:
docs = docs[:100]

In [56]:
import requests
for i in range(len(docs)):
    r = requests.get("http://127.0.0.1:6263/preprocess/chinese?text="+docs[i]).json()
    docs[i] = r['document_topic']

In [57]:
embeddings = list()
for d in docs:
    embeddings.append(cn_dense_retrieval_model.encode([d])[0])

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [58]:
from bertopic import BERTopic
import numpy as np
#topic_model = BERTopic(language="multilingual", calculate_probabilities=True, vectorizer_model=vectorizer_model, verbose=True)
#topics, probs = topic_model.fit_transform(docs[:200])
topic_model = BERTopic(language=None, embedding_model=cn_dense_retrieval_model, calculate_probabilities=False, verbose=True, vectorizer_model=vectorizer_model)#nr_topics='auto')
topics, probs = topic_model.fit_transform(documents=docs, embeddings=np.array(embeddings))

2022-01-07 01:50:04,609 - BERTopic - Reduced dimensionality with UMAP
2022-01-07 01:50:04,615 - BERTopic - Clustered UMAP embeddings with HDBSCAN


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [59]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,0,49,0_房间_在家里_念_影片
1,1,22,1_听_参加_节目_演唱会
2,-1,17,-1_追_商业_在家里_周刊
3,2,12,2_比赛_围棋_运动_篮球


In [60]:
topics

[-1,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 -1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 -1,
 2,
 2,
 2,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 -1,
 -1,
 0,
 -1,
 -1,
 -1,
 2,
 2,
 2,
 2,
 -1,
 -1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 -1,
 -1,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 -1,
 -1]

-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [61]:
topic_model.get_topic(-1)  # Select the most frequent topic

[('追', 0.29651044893759126),
 ('商业', 0.2278249714781895),
 ('在家里', 0.1499236421283187),
 ('周刊', 0.14046181755324083),
 ('大陆', 0.14046181755324083),
 ('剧', 0.14046181755324083),
 ('直播', 0.14046181755324083),
 ('台剧', 0.14046181755324083),
 ('房间', 0.13326545966961664),
 ('综艺节目', 0.12483581666031988)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [63]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [10]:
topic_model.visualize_distribution(probs[0], min_probability=0.015)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [13]:
topic_model.visualize_barchart(top_n_topics=8)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [12]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

ValueError: Make sure to set `n_clusters` lower than the total number of unique topics.

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [13]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [ ]:
topic_model.update_topics(docs, topics, n_gram_range=(1, 2))

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('space', 0.011119596146117955),
 ('nasa', 0.0047697533973351915),
 ('shuttle', 0.0044533985251824495),
 ('orbit', 0.004129278694477752),
 ('spacecraft', 0.004011023125258004),
 ('satellite', 0.003783732360211832),
 ('moon', 0.003639954930862572),
 ('lunar', 0.0034753177228921146),
 ('the moon', 0.002821040122532999),
 ('mars', 0.0028033947303940923)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
new_topics, new_probs = topic_model.reduce_topics(docs, topics, probs, nr_topics=60)

2021-10-17 06:05:02,666 - BERTopic - Reduced number of topics from 220 to 61


# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [29]:
similar_topics, similarity = topic_model.find_topics("我什麼時候去看電影", top_n=5); similar_topics

[-1]

In [30]:
topic_model.get_topic(0)

False

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [ ]:
# Save model
topic_model.save("my_model")	

In [ ]:
# Load model
my_model = BERTopic.load("my_model")	

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
